In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######
housing_data = datasets.fetch_california_housing(data_home='./')

### Preprocessing

housing_data_scaled = preprocessing.scale(housing_data.data)
for idx in range(housing_data.data.shape[1]):
    housing_data_scaled[1] = housing_data_scaled.mean(axis=0)
    
### TF

pred = tf.Variable(tf.random_uniform([1], -1.0, 1.0)) # bias 
for idx in range(housing_data.data.shape[1]): # weight * x
    pred  += (  tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='W_'+str(idx)) * housing_data_scaled[:, idx]) 

loss = tf.reduce_mean(tf.square(pred - housing_data.target)) 
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
init = tf.global_variables_initializer()

### run Session

with tf.Session() as sess:
    sess.run(init)
#     for step in range(10000):
#         sess.run(train)
#         if step % 100 == 0:
#             print(step, sess.run(loss))
    show_graph(tf.get_default_graph().as_graph_def())